<a href="https://colab.research.google.com/github/nom2025/nnj_pub/blob/main/%E3%82%B9%E3%82%B3%E3%82%A2r2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def calculate_daily_score(df):
    """日毎に地震データに基づいてスコアを計算する関数。

    Args:
        df: 地震データを含むpandas DataFrame。

    Returns:
        日毎のスコアを含むpandas DataFrame。
    """

    # 基準日を設定
    df['date'] = pd.to_datetime(df['date'])
    base_date = pd.to_datetime('2010-02-01')
    df['days_since_base'] = (df['date'] - base_date).dt.days

    # 日毎のスコアを格納するDataFrameを作成
    daily_scores = pd.DataFrame(index=df['date'].unique(), columns=['score'])
    daily_scores['score'] = 0  # スコアを初期化
    daily_scores.index.name = 'date' # index名をdateに変更

    # 条件1〜6を日毎に適用し、スコアを計算
    for day in daily_scores.index:
        daily_data = df[df['date'] == day]  # date列で絞り込み

        # 条件1: 1日あたりのmagnitudeデータ数が3回以内が3日以上連続している場合
        if daily_data['magnitude'].count() <= 3:
            previous_day = day - pd.Timedelta(days=1)
            previous_2_day = day - pd.Timedelta(days=2)

            if previous_day in df['date'].values and previous_2_day in df['date'].values and df[df['date'] == previous_day]['magnitude'].count() <= 3 and df[df['date'] == previous_2_day]['magnitude'].count() <= 3:
                daily_scores.loc[day, 'score'] += 1

        # 条件2: 1日あたりのmagnitudeデータ数が3回以内が6日以上連続している場合
        if daily_data['magnitude'].count() <= 3:
            consecutive_days = 0
            for i in range(6):
                target_day = day - pd.Timedelta(days=i)
                if target_day in df['date'].values and df[df['date'] == target_day]['magnitude'].count() <= 3:
                    consecutive_days += 1
                else:
                    break
            if consecutive_days >= 6:
                daily_scores.loc[day, 'score'] += 2

        # 条件3: 1日あたりのmagnitudeデータ数が10回以上で、直前30日間のmagnitudeデータが6.5以下の場合
        if daily_data['magnitude'].count() >= 10:
            previous_30_days_data = df[(df['date'] >= day - pd.Timedelta(days=30)) & (df['date'] < day)]['magnitude']
            if all(previous_30_days_data <= 6.5):
                daily_scores.loc[day, 'score'] += 1

         # 条件4: magnitudeデータが5以上が1日に5回以上ある場合
        if daily_data[daily_data['magnitude'] >= 5]['magnitude'].count() >= 5:
           daily_scores.loc[day, 'score'] += 0.5

        # 条件5: magnitudeデータが5以上が1日に10回以上ある場合
        if daily_data[daily_data['magnitude'] >= 5]['magnitude'].count() >= 10:
            daily_scores.loc[day, 'score'] += 1

        # 条件6: placeに特定の地域が含まれる場合
        target_places = ['日向灘', '択捉島南東沖', '薩摩半島西方沖', '小笠原諸島西方沖', '千島列島', '鳥島近海', '父島近海']
        for place in target_places:
            if daily_data['place'].str.contains(place).any():
                daily_scores.loc[day, 'score'] += 1

    return daily_scores

# CSVファイルを読み込む
df = pd.read_csv('2010-all.csv')

# 日毎のスコアを計算する
daily_scores = calculate_daily_score(df)

# 結果をCSVファイルに出力する
daily_scores.to_csv('japan_scores.csv', index=True)  # index=Trueで日付を出力

print("日毎のスコアを 'japan_scores.csv' に出力しました。")



# ★★★ 世界
def calculate_score(df, base_date):
    """基準日から30日前までのdepthの値に基づいてスコアを計算する関数。

    Args:
        df: データフレーム。'date'列と'depth'列を含む。
        base_date: 基準日を表す文字列（YYYY-MM-DD形式）。

    Returns:
        スコア。
    """

    # 基準日から30日前までのデータを抽出
    end_date = pd.to_datetime(base_date)
    start_date = end_date - pd.DateOffset(days=30)
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    # スコアを初期化
    score = 0

    # magnitude_wの値に基づいてスコアを加算
    for magnitude_w in filtered_df['magnitude_w']:
        if magnitude_w >= 6:
            score += 1/20
        if magnitude_w >= 7:
            score += 2/20
        if magnitude_w >= 8:
            score += 3/20

    return score

# CSVファイルからデータを読み込む
df = pd.read_csv('world-all.csv', parse_dates=['date'])  # 'date'列を日付型に変換

# 基準日を2010年2月1日に設定し、日毎に計算
start_date = pd.to_datetime('2010-02-01')
end_date = df['date'].max()  # データの最終日を取得

# 結果を格納するリストを初期化
results = []

for base_date in pd.date_range(start_date, end_date):
    score = calculate_score(df, base_date.strftime('%Y-%m-%d'))
    # print(f'基準日: {base_date.strftime("%Y-%m-%d")}, スコア: {score}')
    # 結果をリストに追加
    results.append([base_date.strftime("%Y-%m-%d"), score])

# リストからデータフレームを作成
result_df = pd.DataFrame(results, columns=['基準日', 'スコア'])

# 結果をCSVファイルに出力
result_df.to_csv('world_scores.csv', index=False)
print('world_scores.csv に結果が出力されました。')



# ★★★ 深さ
def calculate_score_d(df, base_date):
    """基準日から30日前までのdepthの値に基づいてスコアを計算する関数。

    Args:
        df: データフレーム。'date'列と'depth'列を含む。
        base_date: 基準日を表す文字列（YYYY-MM-DD形式）。

    Returns:
        スコア。
    """

    # 基準日から30日前までのデータを抽出
    end_date = pd.to_datetime(base_date)
    start_date = end_date - pd.DateOffset(days=30)
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    # スコアを初期化
    score = 0

    # depthの値に基づいてスコアを加算
    for depth in filtered_df['depth']:
        if depth >= 300:
            score += 1/20
        if depth >= 400:
            score += 2/20
        if depth >= 500:  # depthが500以上の場合はスコアに3を加算
            score += 3/20

    return score

# CSVファイルからデータを読み込む
df = pd.read_csv('depth.csv', parse_dates=['date'])  # 'date'列を日付型に変換

# 基準日を2010年2月1日に設定し、日毎に計算
start_date = pd.to_datetime('2010-02-01')
end_date = df['date'].max()  # データの最終日を取得

# 結果を格納するリストを初期化
results = []

for base_date in pd.date_range(start_date, end_date):
    score = calculate_score_d(df, base_date.strftime('%Y-%m-%d'))
    # print(f'基準日: {base_date.strftime("%Y-%m-%d")}, スコア: {score}')
    # 結果をリストに追加
    results.append([base_date.strftime("%Y-%m-%d"), score])

# リストからデータフレームを作成
result_df = pd.DataFrame(results, columns=['基準日', 'スコア'])

# 結果をCSVファイルに出力
result_df.to_csv('depth_scores.csv', index=False)
print('depth_scores.csv に結果が出力されました。')



# ★★★ 気圧
# atmospheric.csv ファイルを読み込みます
df = pd.read_csv('atmospheric.csv', parse_dates=['date'])

# 2023年1月2日を基準日として設定します
base_date = pd.to_datetime('2023-01-02')

# スコアを格納するための新しい列を作成します
df['score'] = 0

# 各行を反復処理してスコアを計算します
for index, row in df.iterrows():
    # 現在の日付と基準日との日数差を計算します
    days_diff = (row['date'] - base_date).days

    # 基準日以降の場合
    if days_diff >= 0:
        # 過去30日間のデータを取得
        past_30_days = df[(df['date'] >= row['date'] - pd.Timedelta(days=30)) & (df['date'] < row['date'])]

        # atmospheric の値が5以上であればスコアに1を加算
        if any(past_30_days['atmospheric'] >= 5):
            df.loc[index, 'score'] += float(1/4)

        # atmospheric の値が-5以下であればスコアに1を加算
        if any(past_30_days['atmospheric'] <= -5):
            df.loc[index, 'score'] += float(1/4)

# 結果を atmospheric_scores.csv ファイルに保存します
df.to_csv('atmospheric_scores.csv', index=False)



# ★★★ total

# 4つのCSVファイルを読み込む
atmospheric_scores = pd.read_csv('atmospheric_scores.csv', parse_dates=['date'])
depth_scores = pd.read_csv('depth_scores.csv', parse_dates=['基準日'])
world_scores = pd.read_csv('world_scores.csv', parse_dates=['基準日'])
japan_scores = pd.read_csv('japan_scores.csv', parse_dates=['date'])

# depth_scoresとworld_scoresの列名を変更
depth_scores = depth_scores.rename(columns={'基準日': 'date', 'スコア': 'depth_score'})
world_scores = world_scores.rename(columns={'基準日': 'date', 'スコア': 'world_score'})

# japan_scoresの列名を変更
japan_scores = japan_scores.rename(columns={'score': 'japan_score'})


# 4つのDataFrameをdateをキーにして結合
merged_df = pd.merge(atmospheric_scores[['date', 'score']], depth_scores[['date', 'depth_score']], on='date', how='outer')
merged_df = pd.merge(merged_df, world_scores[['date', 'world_score']], on='date', how='outer')
merged_df = pd.merge(merged_df, japan_scores[['date', 'japan_score']], on='date', how='outer')

# 結合後のDataFrameで欠損値を0で埋める
merged_df = merged_df.fillna(0)

# score列を合計してtotal_score列を作成
merged_df['total_score'] = merged_df['score'] + merged_df['depth_score'] + merged_df['world_score'] + merged_df['japan_score']

# dateとtotal_score列のみを含むDataFrameを作成
total_df = merged_df[['date', 'total_score']]

# total.csvとして出力
total_df.to_csv('total.csv', index=False)

print("total.csv が出力されました。")


/tmp/ipython-input-6-2970106647.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  daily_scores.loc[day, 'score'] += 0.5


日毎のスコアを 'japan_scores.csv' に出力しました。
world_scores.csv に結果が出力されました。
depth_scores.csv に結果が出力されました。


/tmp/ipython-input-6-2970106647.py:220: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'score'] += float(1/4)


total.csv が出力されました。
